## Text2Sound (1)

this method runs in a virtual environment Bark.\
the model is installed from the bark github.

In [ ]:
# import bark
from bark import SAMPLE_RATE, generate_audio, preload_models
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio

## Download and load all models

In [5]:
!unset LD_LIBRARY_PATH
!echo LD_LIBRARY_PATH

LD_LIBRARY_PATH


In [6]:
preload_models()

## Generate audio from text

In [40]:
text_prompt = """
     bonjour ronny et sonja, ici l'assistente artificielle d André.
     André vous invite à déjeuner au restaurant en décembre.
"""
# audio_array = generate_audio(text_prompt,history_prompt="v2/en_speaker_0")
audio_array = generate_audio(text_prompt,history_prompt="v2/fr_speaker_5")
# audio_array = generate_audio(text_prompt,history_prompt="v2/en_speaker_7")


  0%|          | 0/768 [00:00<?, ?it/s]

100%|██████████| 34/34 [00:10<00:00,  3.18it/s]


## Save audio to disc

In [41]:
write_wav("bark_generation.wav", SAMPLE_RATE, audio_array)

## Play text

In [42]:
Audio(audio_array, rate=SAMPLE_RATE)

## Generate audio from text (2)

In [29]:
comm_link = """
    The communication link consists of 2 parts.
    First short distance WiFi, secondly a long distance skyfroid radio link 
"""
drone_part = """
    the drone parts are. A holybro frame, a pixhawk flight controller, an oak D camera,
    a raspberry pi 4 and 8 sensors.      
"""
intro = """
    this video summarizes the hardware and software of my tracking drone.
    The comments are generated by text2audio from suno ai      
"""
soft = """
    we have 2 major software packages. 
    an Ardu pilot firmware for flight controlling, and a custom python code on RPi4 for tracking.

    """
flight = """
    the drone is flashed with Ardu pilot firmware. the groundstation Q groundcontrol enables a full control of the drone.
    Skydroid T12 enables the radio link

    """

text_prompt = flight
audio_array = generate_audio(text_prompt,history_prompt="v2/en_speaker_0")
# audio_array = generate_audio(text_prompt,history_prompt="v2/en_speaker_7")
# audio_array = generate_audio(text_prompt,history_prompt="v2/en_speaker_6")


/home/avncalst/bark/lib/python3.10/site-packages/bark/generation.py:175: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with InferenceContext(), torch.inference_mode(), torch.no_grad(), autocast():
100%|██████████| 25/25 [00:14<00:00,  1.76it/s]


In [33]:
write_wav("flight.wav", SAMPLE_RATE, audio_array)

In [32]:
Audio(audio_array, rate=SAMPLE_RATE)

## Text2sound (2)

this method uses the transformers python module\
this method is slower than the first model

## Download model

In [12]:
from transformers import AutoProcessor, BarkModel
from scipy.io.wavfile import write as write_wav
import numpy

processor = AutoProcessor.from_pretrained("suno/bark")
model = BarkModel.from_pretrained("suno/bark")

/home/avncalst/.local/lib/python3.10/site-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


## Generate audio

In [13]:
voice_preset = "v2/en_speaker_0"
text = """
    The communication link consists of 2 parts.
    First short distance WiFi, secondly a long distance skyfroid radio link 
"""
inputs = processor(text, voice_preset=voice_preset, return_tensors="pt")

# add attention mask here
attention_mask = inputs["attention_mask"]

audio_array = model.generate(
input_ids=inputs["input_ids"],
attention_mask=attention_mask,
pad_token_id=processor.tokenizer.pad_token_id
)

audio_array = audio_array.cpu().numpy().squeeze()

## Save audio

In [14]:
# save audio to disk, but first take the sample rate from the model config
sample_rate = model.generation_config.sample_rate
write_wav("bark_generation_result.wav", sample_rate, audio_array)

In [15]:
from IPython import display
display.Audio(audio_array, rate=sample_rate)